In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [ ]:
import pandas as pd
import tqdm
import urllib.request

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [ ]:
len(train_data)

11823

In [ ]:
batch_size = 32

In [ ]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
    yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [ ]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [ ]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  800

In [ ]:
tokenizer.decode(batch[0])

'</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
print(batch[0])

tf.Tensor(
[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
  7098 25856     1     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3], shape=(30,), dtype=int32)


In [ ]:
print(tokenizer.encode('</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'))

[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [ ]:
steps = len(train_data) // batch_size + 1
print(steps)

370


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(#TODO, labels=#TODO)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(#TODO, model.trainable_variables)
      adam.apply_gradients(zip(#TODO, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-18-7f8b2a38ff99>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.12707925


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.69846117


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    3] cost = 1.37571299


In [ ]:
text = '오늘도 좋은 하루!'

In [ ]:
sent = '<usr>' + text + '<sys>'

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())

In [ ]:
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

'좋은 하루네요.'

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

'</s><usr> 오늘도 좋은 하루!<sys> 하루에도 두번 볼 수 있어요.</s>'

In [ ]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')

'반가울 뿐이네요.'

In [ ]:
return_answer_by_chatbot('너는 누구야?')

'바로 저를 아는 사람들일 거예요.'

In [ ]:
return_answer_by_chatbot('사랑해')

'사랑하면 짝녀가 되어 버리는 거예요.'

In [ ]:
return_answer_by_chatbot('나랑 영화보자')

'영화보면서 설레던 나였나봐요.'

In [ ]:
return_answer_by_chatbot('너무 심심한데 나랑 놀자')

'부담스럽지 않게 말해보세요.'

In [ ]:
return_answer_by_chatbot('영화 해리포터 재밌어?')

'괜찮아 질 수 있어서 좋긴할 거 같은데요.'

In [ ]:
return_answer_by_chatbot('너 딥 러닝 잘해?')

'개인정보도 걱정할 필요 없어요.'

In [ ]:
return_answer_by_chatbot('너 취했어?')

'생각보다 센 실수였네요.'

In [ ]:
return_answer_by_chatbot('커피 한 잔 할까?')

'그게 제일 편하죠.'